In [1]:
# Dependencies and Setup
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import re
import time 
import datetime 
import numpy as np
from config1 import username1,password1,host1,port1,database1
from config2 import username2,password2,host2,port2,database2
import sqlalchemy
from sqlalchemy import create_engine, func, inspect, desc
pd.options.mode.chained_assignment = None  # default='warn'
import requests
import json
from pandas.io.json import json_normalize

In [2]:
# Define Urls for the Johns Hopkins Data
confirm_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv'
death_url='https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv'

# Read in COVID-19 Files
confirm_df=pd.read_csv(confirm_url, error_bad_lines=False)
death_df=pd.read_csv(death_url, error_bad_lines=False)
# Rename column to match orders data
confirm_df=confirm_df.rename(columns={'Province_State': 'State'})
death_df=death_df.rename(columns={'Province_State': 'State'})

# reindex
confirm_df=confirm_df.reset_index()
death_df=death_df.reset_index()
confirm_df=confirm_df.drop(columns=['index'])
death_df=death_df.drop(columns=['index'])

# Drop territories
death_df.dropna(subset = ["Admin2"], inplace=True)
confirm_df.dropna(subset = ["Admin2"], inplace=True)

# Combine State Data
state_confirms=confirm_df.groupby('State').sum()
state_deaths=death_df.groupby('State').sum()

In [3]:
# Clean Confirm data Rows - Remove data for territories and cruise ships, format FIPS
confirm_df=confirm_df[confirm_df.Admin2 != 'Unassigned']
confirm_df=confirm_df.dropna()
confirm_df=confirm_df[~confirm_df['Admin2'].astype(str).str.startswith('Out of')]
confirm_df=confirm_df[confirm_df.Admin2 != 'Out of*']
confirm_df=confirm_df.reset_index()
confirm_df['FIPS']=confirm_df.FIPS.map('{0:0>5.0f}'.format)

# Clean Death Data Rows - Remove data for territories and cruise ships
death_df=death_df[death_df.Admin2 != 'Unassigned']
death_df=death_df.dropna()
death_df=death_df[~death_df['Admin2'].astype(str).str.startswith('Out of')]
death_df=death_df[death_df.Admin2 != 'Out of*']
death_df=death_df.reset_index()
death_df['FIPS']=death_df.FIPS.map('{0:0>5.0f}'.format)

# Pull County data
county_population=death_df[['FIPS','Population']]
county_population=county_population.rename(columns={'FIPS':'fips','Population':'population'})
county_cases=confirm_df.drop(columns=['UID','index','Country_Region','Lat','Long_','iso2','iso3','code3','Combined_Key'])
county_deaths=death_df.drop(columns=['UID','index','Country_Region','Lat','Long_','iso2','iso3','code3','Combined_Key','Population'])
county_cases=county_cases.rename(columns={'Admin2':'County'})
county_deaths=county_deaths.rename(columns={'Admin2':'County'})

last_date=county_cases.columns[-1]

In [4]:
#Find daily totals - county cases
county_cases_daily=county_cases.copy()
[r,c]=county_cases_daily.shape
for j in range (0,r):
    last=0
    for i in range (3,c):
        current=county_cases_daily.iloc[j,i]-last
        last=county_cases_daily.iloc[j,i]
        county_cases_daily.iat[j,i]=current
        

#Find daily totals - county deaths
county_deaths_daily=county_deaths.copy()
[r,c]=county_deaths_daily.shape
for j in range (0,r):
    last=0
    for i in range (3,c):
        current=county_deaths_daily.iloc[j,i]-last
        last=county_deaths_daily.iloc[j,i]
        county_deaths_daily.iat[j,i]=current

In [5]:
# Isolate population data
state_deaths=state_deaths.reset_index()
state_cases=state_confirms.reset_index()
state_population=state_deaths[['State','Population']]
# Format state data
state_cases=state_cases.drop(columns=['UID','FIPS','code3','Lat','Long_'])
state_deaths=state_deaths.drop(columns=['UID','FIPS','code3','Lat','Long_','Population'])


#Find daily totals - state cases
state_cases_daily=state_cases.copy()
[r,c]=state_cases_daily.shape
for j in range (0,r):
    last=0
    for i in range (2,c):
        current=state_cases_daily.iloc[j,i]-last
        last=state_cases_daily.iloc[j,i]
        state_cases_daily.iat[j,i]=current

#Find daily totals - state deaths
state_deaths_daily=state_deaths.copy()
[r,c]=state_deaths_daily.shape
for j in range (0,r):
    last=0
    for i in range (2,c):
        current=state_deaths_daily.iloc[j,i]-last
        last=state_deaths_daily.iloc[j,i]
        state_deaths_daily.iat[j,i]=current

In [6]:
# Specify url
url = 'https://www.finra.org/rules-guidance/key-topics/covid-19/shelter-in-place'

# Parse HTML Object 
response = requests.get(url)
soup = bs(response.text, 'lxml')

# Read Tables
tables = pd.read_html(url)
table = tables[0]

# Remove Excess Columns
orders = table[['State', 'Order Date', 'Order Expiration Date']]

# Remove Special Charectors from State names
orders['State'] = [re.sub(r'[^\w]', ' ', state) for state in orders['State']]

# Initialize lists for date formating
od=orders['Order Date']
dates=[]

# Loop through end dates
for date in od:
    # Split words and select formatted dates
    split = date.split()
    res = [i for i in split if '/' in i]
    try:
        # Select the date and reformate into standard form
        res=res[-1]
        splitdate = res.split('/')
        if len(splitdate[2]) == 2:
            splitdate[2]='20'+splitdate[2]
        formated = datetime.date(int(splitdate[2]),int(splitdate[0]),int(splitdate[1]))
        datestr=str(formated)
    except:
        # Add null sets for states without expiration dates
        datestr=np.nan
    # Add formatted dates to list
    dates.append(datestr)
# Add dates back in
orders['Order Date']=dates
    
# Initialize lists for date formating
oed= orders['Order Expiration Date']
dates=[]

# Loop through end dates
for date in oed:
    # Split words and select formatted dates
    split = date.split()
    res = [i for i in split if '/' in i]
    try:
        # Select the date and reformate into standard form
        res=res[-1]
        splitdate = res.split('/')
        if len(splitdate[2]) == 2:
            splitdate[2]='20'+splitdate[2]
        formated = datetime.date(int(splitdate[2]),int(splitdate[0]),int(splitdate[1]))
        datestr=str(formated)
    except:
        # Add null sets for states without expiration dates
        datestr=np.nan
    # Add formatted dates to list
    dates.append(datestr)
# Add dates back in and correct index
orders['Order Expiration Date']=dates

orders.iloc[1,2]=str(datetime.date(int(2020),int(4),int(24)))
orders.iloc[19,2]=str(datetime.date(int(2020),int(6),int(8)))

# Format to merge and Merge
orders['State']=[x.strip() for x in orders['State']]
orders=orders.merge(state_population,on='State',how='right')
# Rename columns to be compatible with sql
orders=orders.rename(columns={'State':'state','Order Date': 'order_date','Order Expiration Date':'order_expiration_date','Population':'population'})
heatmap=orders.copy()
orders=orders.drop(columns=['population'])
orders=orders.sort_values(by=['state'])
orders=orders.reset_index()
orders=orders.drop(columns=['index'])
orders.to_csv(r'Orders.csv')

In [7]:
# Prep data for heatmap
# Test % Positive
state_names=state_cases.iloc[:,0]
state_totals=state_cases.iloc[:,-1]
state_total_deaths=state_deaths.iloc[:,-1]
thisweek_cases=state_cases_daily.iloc[:, -7:].sum(axis=1)
lastweek_cases=state_cases_daily.iloc[:, -14:-7].sum(axis=1)
thisweek_deaths=state_deaths_daily.iloc[:, -7:].sum(axis=1)
lastweek_deaths=state_deaths_daily.iloc[:, -14:-7].sum(axis=1)
combine = pd.DataFrame({'state':state_names,'total_cases':state_totals,'total_deaths':state_total_deaths,'thisweek_cases':thisweek_cases,'lastweek_cases':lastweek_cases,'thisweek_deaths':thisweek_deaths,'lastweek_deaths':lastweek_deaths})
heatmap=heatmap.merge(combine,on='state',how='right')
heatmap['per100k']=(heatmap['total_cases']/(heatmap['population']/100000))
heatmap['state']=heatmap.state.str.title()

url="https://covidtracking.com/api/v1/states/current.json"
response = requests.get(url).json()
test_data=pd.json_normalize(response)
test_data['positive_rate']=(test_data['positive']/test_data['totalTestResults'])*100
test_data=test_data[['state','positive_rate']]
url="https://covidtracking.com/api/v1/states/info.json"
response = requests.get(url).json()
name_data=pd.json_normalize(response)
name_data=name_data[['state','name']]
test_data=test_data.merge(name_data,on='state',how='right')
test_data=test_data.drop(columns=['state'])
test_data=test_data.rename(columns={'name':'state'})
state_heatmap=heatmap.merge(test_data,on='state',how='left')
state_heatmap.sort_values(by=['state'], inplace=True)
state_heatmap=state_heatmap.reset_index()
state_heatmap=state_heatmap.drop(columns=['index'])

In [8]:
usa_heatmap=pd.DataFrame(state_heatmap.sum(axis = 0, skipna = True))
usa_heatmap=usa_heatmap.T
usa_heatmap=usa_heatmap.drop(columns=['state','per100k','positive_rate'])
usa_heatmap=usa_heatmap.astype(int)
url="https://covidtracking.com/api/v1/us/current.json"
response = requests.get(url).json()
temp=pd.json_normalize(response)
usa_heatmap['positive_rate']=(temp['positive']/temp['totalTestResults'])*100
usa_heatmap['per100k']=(usa_heatmap['total_cases']/usa_heatmap['population'])*100000
usa_heatmap['lastdate']=last_date

In [9]:
# Prep data for county heatmaps
# Test % Positive
# Cases per 100k
county_fips=county_cases.iloc[:,0]
county_names=county_cases.iloc[:,1]
state_names=county_cases.iloc[:,2]
county_totals=county_cases.iloc[:,-1]
county_total_deaths=county_deaths.iloc[:,-1]
thisweek_cases=county_cases_daily.iloc[:, -7:].sum(axis=1)
lastweek_cases=county_cases_daily.iloc[:, -14:-7].sum(axis=1)
thisweek_deaths=county_deaths_daily.iloc[:, -7:].sum(axis=1)
lastweek_deaths=county_deaths_daily.iloc[:, -14:-7].sum(axis=1)
county_heatmap=pd.DataFrame({'fips':county_fips,'county':county_names,'state':state_names,'total_cases':county_totals,'total_deaths':county_total_deaths,'thisweek_cases':thisweek_cases,'lastweek_cases':lastweek_cases,'thisweek_deaths':thisweek_deaths,'lastweek_deaths':lastweek_deaths})
county_heatmap=county_heatmap.merge(county_population,on='fips',how='right')
county_heatmap['per100k']=(county_heatmap['total_cases']/(county_heatmap['population']/100000))

In [10]:
# Split Arrays
[county_cases1,county_cases2,county_cases3]=np.array_split(county_cases, 3)
[county_deaths1,county_deaths2,county_deaths3]=np.array_split(county_deaths,3)

In [11]:
# Python SQL toolkit and Object Relational Mapper DB1
connection1=f'{username1}:{password1}@{host1}:{port1}/{database1}'
engine1 = create_engine(f'postgresql://{connection1}')

In [12]:
#Write to DB1 9817/10000 rows
usa_heatmap.to_sql(name='usa_heatmap', con=engine1, if_exists='replace', index=False)

In [13]:
state_heatmap.to_sql(name='state_heatmap', con=engine1, if_exists='replace', index=False)
county_heatmap.to_sql(name='county_heatmap', con=engine1, if_exists='replace', index=False)

In [14]:
county_cases1.to_sql(name='county_cases1', con=engine1, if_exists='replace', index=False)
county_cases2.to_sql(name='county_cases2', con=engine1, if_exists='replace', index=False)
county_cases3.to_sql(name='county_cases3', con=engine1, if_exists='replace', index=False)

In [15]:
county_deaths1.to_sql(name='county_deaths1', con=engine1, if_exists='replace', index=False)
county_deaths2.to_sql(name='county_deaths2', con=engine1, if_exists='replace', index=False)
county_deaths3.to_sql(name='county_deaths3', con=engine1, if_exists='replace', index=False)

In [16]:
state_cases.to_sql(name='state_cases', con=engine1, if_exists='replace', index=False)
state_deaths.to_sql(name='state_deaths', con=engine1, if_exists='replace', index=False)

In [17]:
orders.to_sql(name='orders', con=engine1, if_exists='replace', index=False)

In [18]:
# state_cases_daily.to_sql(name='state_cases_daily', con=engine1, if_exists='replace', index=False)
# state_deaths_daily.to_sql(name='state_deaths_daily', con=engine1, if_exists='replace', index=False)

In [19]:
# # Python SQL toolkit and Object Relational Mapper DB1
# connection2=f'{username2}:{password2}@{host2}:{port2}/{database2}'
# engine2= create_engine(f'postgresql://{connection2}')

In [20]:
# #Write to DB2 6284/10000 rows
# county_cases_daily.to_sql(name='county_cases_daily', con=engine2, if_exists='replace', index=False)
# county_deaths_daily.to_sql(name='county_deaths_daily', con=engine2, if_exists='replace', index=False)